In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
data_dir = '/home/omkar/Projects/panGenome/data/entero_project/'

In [5]:
df_clusters = pd.read_pickle('/home/omkar/Projects/panGenome/data/entero_project/general/df_clusters.p')

In [6]:
df_clusters

product  \
genus        genus_cat    genome_name                                        genome_length patric_id  no_of_clusters_per_genome no_of_records no_of plasmids record_accession record_len no_of_clusters_per_record cluster_id                                       
Klebsiella   Klebsiella   Klebsiella oxytoca KCTC 1686                       5974109       1006551.4  6                         1             NaN            accnCP003218     5974109    6                         1006551.4.cluster001             butyrolactone   
                                                                                                                                                                                                                   1006551.4.cluster002               arylpolyene   
                                                                                                                                                                                                                   1006551.4.cluster003               bacteriocin   
                                                                                                                                                                                                                   1006551.4.cluster004                      nrps   
                                                                                                                                                                                                                   1006551.4.cluster005               thiopeptide   
                                                                                                                                                                                                                   1006551.4.cluster006                t1pks-nrps   
Serratia     Serratia     Serratia plymuthica RVH1                           5514320       1006598.5  11                        1             NaN            c00001_accnAR..  5514320    11                        1006598.5.cluster001               hserlactone   
                                                                                                                                                                                                                   1006598.5.cluster002                      nrps   
                                                                                                                                                                                                                   1006598.5.cluster003   arylpolyene-siderophore   
                                                                                                                                                                                                                   1006598.5.cluster004                      nrps   
                                                                                                                                                                                                                   1006598.5.cluster005                t1pks-nrps   
                                                                                                                                                                                                                   1006598.5.cluster006               thiopeptide   
                                                                                                                                                                                                                   1006598.5.cluster007                   terpene   
                                                                                                                                                                                                                   1006598.5.cluster008                      nrps   
                                                                          

In [7]:
#### Generate dataframe to support Cytoscape visualization
'/home/omkar/Projects/panGenome/data/entero_project/bigscape_as4_no_hybrids/network_files/2019-05-08_14-19-17_glocal_hybrids_off_multi_cutoff/Network_Annotations_Full.tsv'

'/home/omkar/Projects/panGenome/data/entero_project/bigscape_as4_no_hybrids/network_files/2019-05-08_14-19-17_glocal_hybrids_off_multi_cutoff/Network_Annotations_Full.tsv'

In [10]:
# get dictionary with data on all bgcs from bigscape analysis
network_bigscape_dir = os.path.join(data_dir, 'bigscape_as4_no_hybrids/network_files/2019-05-08_14-19-17_glocal_hybrids_off_multi_cutoff/')

net_path = os.path.join(network_bigscape_dir, 'Network_Annotations_Full.tsv')
df_clusters_cyto = pd.read_csv(net_path, sep='\t')
df_clusters_cyto.set_index('BGC',inplace=True)

In [ ]:
idx = pd.IndexSlice
for index in df_clusters_cyto.index:
    if 'BGC' in index:
        df_clusters_cyto.loc[index, 'mibig'] = 'mibig'
        df_clusters_cyto.loc[index, 'patric'] = '#'
        df_clusters_cyto.loc[index, 'Organism'] = df_clusters_cyto.loc[index, 'Description'][:-26]
        df_clusters_cyto.loc[index, 'genus'] = 'MIBIG CLUSTER'
        df_clusters_cyto.loc[index, 'genus_cat'] = 'MIBIG CLUSTER'

    else:
        df_clusters_cyto.loc[index, 'mibig'] = 'TargetCluster'
        
        cluster_series = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,index], idx[:]]
        
        genus = cluster_series.index.get_level_values('genus').tolist()[0]
        genus_cat = cluster_series.index.get_level_values('genus_cat').tolist()[0]
        pat = cluster_series.index.get_level_values('patric_id').tolist()[0]
        org =  cluster_series.index.get_level_values('genome_name').tolist()[0]
        product_cat = cluster_series.product_cat.tolist()[0]

        df_clusters_cyto.loc[index, 'patric'] = pat
        df_clusters_cyto.loc[index, 'Organism'] = org
        df_clusters_cyto.loc[index, 'genus'] = genus
        df_clusters_cyto.loc[index, 'genus_cat'] = genus_cat
        df_clusters_cyto.loc[index, 'product_cat'] = product_cat
        

In [ ]:
# Create union of all network files for one figure
# BIGSCAPE Results parsing for Cytoscape visualization
# loop through all the types of BGCs
bgc_types = [f for f in os.listdir(network_bigscape_dir) if '.tsv' not in f]

df_network_union = pd.DataFrame()
for bgc in bgc_types:
    net_file = bgc + '_c0.50.network'
    df_tmp = pd.read_csv(os.path.join(network_bigscape_dir, bgc, net_file), sep='\t')
    print(df_tmp.shape)
    df_network_union = df_network_union.append(df_tmp, ignore_index = True) 
    print(df_network_union.shape)

In [ ]:
df_network_union_filter_singles = df_network_union[df_network_union['Clustername 1'] != df_network_union['Clustername 2']]
df_network_union_singles = df_network_union[df_network_union['Clustername 1'] == df_network_union['Clustername 2']]

for index in df_clusters_cyto.index:
    if index in df_network_union_singles['Clustername 1'].tolist():
        if 'BGC' in index:
            df_clusters_cyto.loc[index, 'singleton'] = 'Yes'
        else:
            df_clusters_cyto.loc[index, 'singleton'] = 'Yes'
    else:
        if 'BGC' in index:
            df_clusters_cyto.loc[index, 'singleton'] = 'No'
        else:
            df_clusters_cyto.loc[index, 'singleton'] = 'No'    

In [ ]:
for clust_class in df_clusters_cyto['BiG-SCAPE class'].unique():
    df_class = df_clusters_cyto[df_clusters_cyto['BiG-SCAPE class'] == clust_class]
    cluster_ids = df_class.index
    with open(os.path.join(data_dir, 'cytoscape','Input', clust_class + '_cluster_ids.txt'), 'w') as f:
        for item in cluster_ids:
            f.write("%s\n" % item)
#     df_class.to_csv(os.path.join(data_dir, 'cytoscape','Input', clust_class + '_cyto_filter_singles.csv'))

In [ ]:
# Save inputs for cytoscape
df_clusters_cyto.to_pickle(os.path.join(data_dir, 'cytoscape', 'Input','df_clusters_cyto_filter_singles_50.p'))
df_clusters_cyto.to_csv(os.path.join(data_dir, 'cytoscape','Input','df_clusters_cyto_filter_singles_50.csv'))

# df_network_union_filter_singles.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_filter_singles_50.p'))
# df_network_union_filter_singles.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_filter_singles_50.csv'))

# df_network_union_singles.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_singles_50.p'))
# df_network_union_singles.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_singles_50.csv'))

# df_clusters_cyto = pd.read_pickle(os.path.join(data_dir, 'cytoscape','Input','df_clusters_cyto_filter_singles_70.p'))
# df_network_union = pd.read_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_filter_singles_70.p'))

In [ ]:
df_network_union.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union.p'))
df_network_union.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union.csv'))


In [ ]:
# Network propeties from networkx
G_clusters = nx.Graph()
# Create a list of tuples with weighted edges using df_network_union_filter_singles
edge_list = []
for index in df_network_union_filter_singles.index:
    edge_tuple = (df_network_union_filter_singles.loc[index, 'Clustername 1'], df_network_union_filter_singles.loc[index, 'Clustername 2'], 1 - df_network_union_filter_singles.loc[index, 'Raw distance'])
    edge_list.append(edge_tuple)
    
# Create graph of clusters from edges
G_clusters.add_weighted_edges_from(edge_list)

# Find connected components or cluster families
Families_list = list(nx.connected_component_subgraphs(G_clusters))
print('Total families detected: ', len(Families_list))

# Sort families in decreasing order of occurrence
# MIBIG clusters is also taken in to count
family_size = [len(family) for family in Families_list]
orig_index = list(range(len(family_size)))
orig_index_fam_size_dict = dict(zip(orig_index, family_size))

sorted_index_fam_size_dict = OrderedDict(sorted(orig_index_fam_size_dict.items(), key=lambda x: x[1]), )
new_index = list(range(len(family_size)-1,-1,-1))
orig_index_sorted = list(sorted_index_fam_size_dict.keys())
new_orig_index_dict = dict(zip(new_index, orig_index_sorted))

# Ordered family graphs
family_graphs = [Families_list[new_orig_index_dict[fam_id]] for fam_id in range(len(Families_list))]

pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/family_graphs_50.p',"wb")
pickle.dump(family_graphs, pickle_out)
pickle_out.close()

pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/G_clusters_50.p',"wb")
pickle.dump(G_clusters, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/family_graphs.p',"rb")
family_graphs = pickle.load(pickle_out)
pickle_out.close()

pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/G_clusters.p',"rb")
G_clusters = pickle.load(pickle_out)
pickle_out.close()

pickle_out = open('/home/omkar/Projects/panGenome/data/entero_project/general/genus_colors.p',"rb")
my_palette = pickle.load(pickle_out)
pickle_out.close()

In [ ]:
# Nodes in subnetworks with at least 10 clusters
nodes_large_subgraphs = []
nodes_small_subgraphs = []
for subgraph in family_graphs:
    if len(subgraph.node) > 9:
        for node in subgraph.node:
            nodes_large_subgraphs.append(node)
    else:
        for node in subgraph.node:
            nodes_small_subgraphs.append(node)

# Update df_union network accordingly
df_network_union_large = df_network_union_filter_singles
df_network_union_small = df_network_union_filter_singles
idx_large = []
idx_small = []
for df_idx in df_network_union_filter_singles.index:
    if df_network_union_filter_singles.loc[df_idx, 'Clustername 1'] in nodes_large_subgraphs or df_network_union_filter_singles.loc[df_idx, 'Clustername 2'] in nodes_large_subgraphs:
        idx_large.append(df_idx)
#         print(df_idx)
    else:
        idx_small.append(df_idx)
    
df_network_union_large = df_network_union_filter_singles.loc[idx_large,:]
df_network_union_small = df_network_union_filter_singles.loc[idx_small,:]
df_network_union_large.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_large_30.csv'))
df_network_union_small.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_small_30.csv'))

df_network_union_large.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_large_30.p'))
df_network_union_small.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_small_30.p'))

## Supp Figure

In [ ]:
# add colors according to genera for cluster IDs
cluster_list = df_clusters.index.get_level_values(11).tolist()
cluster_color = dict()
cluster_genome = dict()
cluster_genus = dict()
cluster_product = dict()

for cluster_id in cluster_list:
    genus_cat = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,cluster_id], :].index.get_level_values(1).tolist()[0]
    genome_name = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,cluster_id], :].index.get_level_values(2).tolist()[0]
    product_cat = df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,cluster_id], 'product_cat']
    
    cluster_color[cluster_id] = my_palette[genus_cat]
    cluster_genome[cluster_id] = genome_name
    cluster_genus[cluster_id] = genus_cat
    cluster_product[cluster_id] = product_cat

In [ ]:
# Add mibig to genus list as black color
# genus_list.append('MIBIG')
# my_palette['MIBIG'] = (0,0,0)

In [ ]:
os.mkdir('/home/omkar/Projects/panGenome/data/entero_project/families/adj_mat_05/')
os.mkdir('/home/omkar/Projects/panGenome/data/entero_project/families/figures_05/')

In [ ]:
# A = nx.adjacency_matrix(G)
fig_path = '/home/omkar/Projects/panGenome/data/entero_project/families/figures_05/'
csv_path = '/home/omkar/Projects/panGenome/data/entero_project/families/adj_mat_05/'

top8_family_graphs = family_graphs[0:7]
for fam_idx in range(len(top8_family_graphs)):
    G = top8_family_graphs[fam_idx]
    node_colors = []
    cluster_genus_list = []
    cluster_genome_list = []
    
    for node in G.nodes:
        if 'BGC' in node:
            node_colors.append((0,0,0))
            cluster_genus_list.append('MIBIG')
            cluster_genome_list.append('MIBIG')
        else:
            node_colors.append(cluster_color[node])                       
            cluster_genus_list.append(cluster_genus[node])
            cluster_genome_list.append(cluster_genome[node])

    # Plot adjacency matrix
    plt.figure()
    df_g = nx.to_pandas_adjacency(G)
    r = sns.clustermap(df_g, figsize=(20,20), cmap='BuPu', row_colors=node_colors, col_colors=node_colors, xticklabels=False, yticklabels=False)
    for label in genus_list:
        r.ax_col_dendrogram.bar(0, 0, color=my_palette[label],
                            label=label, linewidth=0)
    r.ax_col_dendrogram.legend(loc=(1.1,-4), ncol=1)
        
#     Plot degree distribution   
#     degree_sequence = [d for n, d in G.degree()]  # degree sequence
    
#     plt.axes([1, 0.1, 0.3, 0.3])
#     plt.title("Degree Histogram comp-" +  str(fam_idx+1))
#     plt.ylabel("Count")
#     plt.xlabel("Degree")

#     ax.set_xticklabels(deg)
#     plt.hist(degree_sequence, bins=20)
    
#     # draw graph in inset

#     plt.axes([1.05, 0.2, 0.2, 0.2])
#     Gcc = sorted(nx.connected_component_subgraphs(G), key=len, reverse=True)[0]
#     pos = nx.spring_layout(G)
#     plt.axis('off')
#     nx.draw_networkx_nodes(G, pos, node_size=40, node_color=node_colors)
#     nx.draw_networkx_edges(G, pos, alpha=0.6)
    
#     Network properties
#     cell_text = [['No of nodes', len(G.nodes)], ['No of edges', len(G.edges)], ['Diameter', nx.diameter(G)],
#                 ['Clustering coeff.', round(nx.average_clustering(G),1)], ['Avg. shortest path', round(nx.average_shortest_path_length(G),1)]]
#     plt.axes([1.15, 0.4, 0.2, 0.4])
#     plt.axis('off')
#     table = plt.table(cellText=cell_text, cellLoc='center', colWidths = [.7,.3],
#                     loc='upper left')
#     table.scale(1, 5)
    
    
    r.savefig(os.path.join(fig_path, 'comp_' + str(fam_idx+1)))
   
    df_to_save = r.data2d
    
    cluster_genus_list2 = []
    cluster_genome_list2 = []
    
    for node in df_to_save.index:
        if 'BGC' in node:
            node_colors.append((0,0,0))
            cluster_genus_list2.append('MIBIG')
            cluster_genome_list2.append('MIBIG')
        else:
            node_colors.append(cluster_color[node])                       
            cluster_genus_list2.append(cluster_genus[node])
            cluster_genome_list2.append(cluster_genome[node])
    
    df_to_save['genus'] = cluster_genus_list2
    df_to_save['genome'] = cluster_genome_list2
        
    df_to_save = df_to_save.reindex(columns=['genus' , 'genome'] + r.data2d.columns.tolist())
    df_to_save.to_csv(os.path.join(csv_path, 'comp_' + str(fam_idx+1)))

    plt.show()
    plt.close() 

In [ ]:
# We further analyzed the top 5 subgraphs and traced the evolutions in each subgraph

In [ ]:
# Family 2 NRPSs
for i in range():
    G = top50_family_graphs[i]

    node_colors = []
    cluster_genus_list = []
    cluster_genome_list = []
    
    for node in G.nodes:
        if 'BGC' in node:
            node_colors.append((0,0,0))
            cluster_genus_list.append('MIBIG')
            cluster_genome_list.append('MIBIG')
        else:
            node_colors.append(cluster_color[node])                       
            cluster_genus_list.append(cluster_genus[node])
            cluster_genome_list.append(cluster_genome[node])

    # Plot adjacency matrix
    plt.figure()
    df_g = nx.to_pandas_adjacency(G)
    r = sns.clustermap(df_g, figsize=(20,20), cmap='BuPu', row_colors=node_colors, col_colors=node_colors, xticklabels=False, yticklabels=False)
    for label in genus_list:
        r.ax_col_dendrogram.bar(0, 0, color=my_palette[label],
                            label=label, linewidth=0)
    r.ax_col_dendrogram.legend(loc=(1.1,-4), ncol=1)
        
    # Plot degree distribution   
    degree_sequence = [d for n, d in G.degree()]  # degree sequence
    
    plt.axes([1, 0.1, 0.3, 0.3])
    plt.title("Degree Histogram comp-" +  str(fam_idx+1))
    plt.ylabel("Count")
    plt.xlabel("Degree")

    ax.set_xticklabels(deg)
    plt.hist(degree_sequence, bins=20)
    
    # draw graph in inset

    plt.axes([1.05, 0.2, 0.2, 0.2])
    Gcc = sorted(nx.connected_component_subgraphs(G), key=len, reverse=True)[0]
    pos = nx.spring_layout(G)
    plt.axis('off')
    nx.draw_networkx_nodes(G, pos, node_size=40, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    
    # Network properties
    cell_text = [['No of nodes', len(G.nodes)], ['No of edges', len(G.edges)], ['Diameter', nx.diameter(G)],
                ['Clustering coeff.', round(nx.average_clustering(G),1)], ['Avg. shortest path', round(nx.average_shortest_path_length(G),1)]]
    plt.axes([1.15, 0.4, 0.2, 0.4])
    plt.axis('off')
    table = plt.table(cellText=cell_text, cellLoc='center', colWidths = [.7,.3],
                    loc='upper left')
    table.scale(1, 5)

In [ ]:
fam_mibig_dict = dict(zip(df_clusters.family_id, df_clusters.mibig_ids))
fam_mibig_dict

mibig_fam_dict = dict(zip(df_clusters.mibig_ids, df_clusters.family_id))
family_graphs[1].nodes

## Supp Figure

In [ ]:
# Assign family ID in df_clusters
# for fam_id in range(len(family_graphs)):
#     cluster_set = family_graphs[fam_id]
#     clusters_list = list(cluster_set)
#     mibig_ids = []
#     for cluster in clusters_list:
#         if 'BGC' not in cluster:
#             df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,:,cluster], 'family_id'] = str(fam_id + 1)
#             print(fam_id)
            
#         else:
#             mibig_ids.append(cluster)
#     if mibig_ids:
#         for cluster in clusters_list:
#             if 'BGC' not in cluster:
#                 df_clusters.loc[idx[:,:,:,:,:,:,:,:,:,:,:,:,cluster], 'mibig_ids'] = str(mibig_ids)
        
# df_clusters.family_id.fillna('singleton', inplace=True)
# df_clusters.mibig_ids.fillna('no_mibig', inplace=True)


# Generate figure for genera distribution and number of clusters per genome
df_genome_data = df_entero
# df_genome_data = df_genome_data.set_index('Strain_ID')
extracted_columns = ['Genus', 'Genome Length', 'GC Content']

df_genome_data = df_genome_data[extracted_columns]

fam_columns = []
for tmp_id in range(len(Families_list)):
    fam_columns.append(str(tmp_id + 1))
fam_columns.append('singleton')
# fam_columns.append('Other')

for col in fam_columns:
    df_genome_data.loc[:,col] = 0
    
for pat_id in df_genome_data.index:
    genus_name = pat_genus_dict[pat_id]
    if genus_name in abundant_genera:
        df_genome_data.loc[pat_id, 'genus_cat'] = genus_name
    else:
        df_genome_data.loc[pat_id, 'genus_cat'] = 'Other'
        
    if pat_id in df_clusters.index.get_level_values(4).tolist():
        no_of_clusters = df_clusters.loc[idx[:,:,:,:,pat_id,:,:,:,:,:,:,:],:].index.get_level_values(5).tolist()[0]
        df_genome_data.loc[pat_id, 'no_of_clusters'] = no_of_clusters
        fam_list = df_clusters.loc[idx[:,:,:,:,pat_id,:,:,:,:,:,:,:],'family_id'].tolist()
        for fam in fam_list:
            df_genome_data.loc[pat_id, fam] = df_genome_data.loc[pat_id, fam] + 1
    else:
        df_genome_data.loc[pat_id, 'no_of_clusters'] = 0
        
# Reindexing
df_genome_data = df_genome_data.reindex(columns= extracted_columns + ['genus_cat', 'no_of_clusters'] + fam_columns)
df_genome_data.sort_values('genus_cat', inplace=True)

# Colors according to genera
# Prepare a vector of color mapped to the 'genus' column
my_palette = dict(zip(df_genome_data.genus_cat.unique(), sns.color_palette("tab20", len(df_genome_data.genus_cat.unique()))))
row_colors = df_genome_data.genus_cat.map(my_palette)

genus_list = list(abundant_genera)
genus_list.append('Other')

genome_product_map = df_genome_data.iloc[:,6:]

In [ ]:
dict_rgb_pallete = dict()
for genus in my_palette.keys():
    color_values = my_palette[genus]
    print(genus)
    for color in color_values:
        print(color*255)

In [ ]:
for pat_id in df_genome_data.index:
    genus_name = pat_genus_dict[pat_id]
    if genus_name in abundant_genera:
        df_genome_data.loc[pat_id, 'genus_cat'] = genus_name
    else:
        df_genome_data.loc[pat_id, 'genus_cat'] = 'Other'
        
    if pat_id in df_clusters.index.get_level_values(4).tolist():
        no_of_clusters = df_clusters.loc[idx[:,:,:,:,pat_id,:,:,:,:,:,:,:],:].index.get_level_values(5).tolist()[0]
        df_genome_data.loc[pat_id, 'no_of_clusters'] = no_of_clusters
        fam_list = df_clusters.loc[idx[:,:,:,:,pat_id,:,:,:,:,:,:,:],'family_id'].tolist()
        for fam in fam_list:
            df_genome_data.loc[pat_id, fam] = df_genome_data.loc[pat_id, fam] + 1
    else:
        df_genome_data.loc[pat_id, 'no_of_clusters'] = 0
        
# Reindexing
df_genome_data = df_genome_data.reindex(columns= extracted_columns + ['genus_cat', 'no_of_clusters'] + fam_columns)
df_genome_data.sort_values('genus_cat', inplace=True)

# Colors according to genera
# Prepare a vector of color mapped to the 'genus' column
my_palette = dict(zip(df_genome_data.genus_cat.unique(), sns.color_palette("tab20", len(df_genome_data.genus_cat.unique()))))
row_colors = df_genome_data.genus_cat.map(my_palette)

genus_list = list(abundant_genera)
genus_list.append('Other')

In [ ]:
# Colors according to products
# Prepare a vector of color mapped to the 'product' column
my_palette2 = dict(zip(df_clusters.product_cat.unique(), sns.color_palette("tab20", len(df_clusters.product_cat.unique()))))
fam_to_prod_dict = dict(zip(df_clusters['product_cat'].tolist(), df_clusters['family_id'].tolist()))
col_colors = pd.Series(fam_to_prod_dict.values()).map(my_palette2)

In [ ]:
genome_product_map = df_genome_data.iloc[:,5:]

reorder_index = []
for genus in genus_list:
    genus_block_ids = df_genome_data[df_genome_data.genus_cat == genus].index
    df_genus_block = genome_product_map.loc[genus_block_ids,:]
    df_genus_block = df_genus_block.loc[:,df_genus_block.columns[df_genus_block.sum(axis=0) > 0]]
    r_block = sns.clustermap(df_genus_block, cmap=sns.color_palette("CMRmap_r",10), col_cluster=False, row_cluster=True, 
              linewidths=0, yticklabels=False)
    reorder_index = reorder_index + r_block.data2d.index.tolist()
    
reorder_columns = []

genome_product_map = genome_product_map.reindex(index=reorder_index)

mibig_fam_list =[]
no_mibig_fam_list = []
for fam_id in fam_mibig_dict.keys():
    if fam_mibig_dict[fam_id] == 'no_mibig':
        no_mibig_fam_list.append(fam_id)
    else:
        mibig_fam_list.append(fam_id)
        
df_genus_block = genome_product_map.loc[:,mibig_fam_list]
df_genus_block = df_genus_block.loc[:,df_genus_block.columns[df_genus_block.sum(axis=0) > 0]]
r_block = sns.clustermap(df_genus_block, cmap=sns.color_palette("CMRmap_r",10), col_cluster=True, row_cluster=False, 
              linewidths=0, yticklabels=False)
reorder_columns = reorder_columns + r_block.data2d.columns.tolist()

df_genus_block = genome_product_map.loc[:,no_mibig_fam_list]
df_genus_block = df_genus_block.loc[:,df_genus_block.columns[df_genus_block.sum(axis=0) > 0]]
r_block = sns.clustermap(df_genus_block, cmap=sns.color_palette("CMRmap_r",10), col_cluster=True, row_cluster=False, 
              linewidths=0, yticklabels=False)
reorder_columns = reorder_columns + r_block.data2d.columns.tolist()

In [ ]:
df_genus_block = genome_product_map.loc[:,no_mibig_fam_list]
df_genus_block = df_genus_block.loc[:,df_genus_block.columns[df_genus_block.sum(axis=0) > 0]]
r_block = sns.clustermap(df_genus_block, cmap=sns.color_palette("tab20", 16), col_cluster=True, row_cluster=False, 
              linewidths=0, yticklabels=False)


In [ ]:
## Supp Figure

In [ ]:
genome_product_map = df_genome_data.iloc[:,5:]

genome_product_map = genome_product_map.reindex(index=reorder_index, columns=reorder_columns)
genome_product_map_reduced = genome_product_map.loc[:,genome_product_map.sum(axis=0) > 4]
genome_product_map_reduced = genome_product_map_reduced.drop(columns='singleton')
sns.set_context("poster", font_scale=1.2)
g = sns.clustermap(genome_product_map_reduced, cmap=sns.color_palette("CMRmap_r"), col_cluster=False, 
                   row_cluster=False, row_colors=row_colors, figsize = (60,30),linewidths=0, yticklabels=False)

for label in genus_list:
    g.ax_col_dendrogram.bar(0, 0, color=my_palette[label],
                            label=label, linewidth=0)
    
g.ax_col_dendrogram.legend(loc=(1.1,-10), ncol=1)

ax = g.ax_heatmap
ax.set_xlabel("Families of Gene clusters (>5 members)", fontsize=70)
ax.set_ylabel("List of genomes", fontsize=70)

In [ ]:
# Generate dataframe for family

In [ ]:

df_families = pd.DataFrame(index=df_genome_data.columns[5:], columns= ['No_of_clusters', 'mibig'] + abundant_genera + ['Target_clusters'])

# Assign family ID in df_clusters
for fam_id in new_orig_index_dict.keys():
    orig_id = new_orig_index_dict[fam_id]
    cluster_set = Families_list[orig_id]
    clusters_list = list(cluster_set)
    mibig_ids = []
    tgt_clusters = []
    for cluster in clusters_list:
        if 'BGC' not in cluster:
            tgt_clusters.append(cluster)
        else:
            mibig_ids.append(cluster)
    df_families.loc[str(fam_id + 1), 'No_of_clusters'] = len(tgt_clusters)
    df_families.loc[str(fam_id + 1), 'Target_clusters'] = str(tgt_clusters)
    df_families.loc[str(fam_id + 1),'mibig'] = str(mibig_ids)

In [ ]:
fam_gen_cnt = df_clusters.groupby(['family_id','genus_cat']).count()
for fam_idx in df_families.index:
    if fam_idx in fam_gen_cnt.index:
        gen_tmp_df = fam_gen_cnt.loc[fam_idx]
        for gen_idx in gen_tmp_df.index:
            df_families.loc[fam_idx, gen_idx] = gen_tmp_df.loc[gen_idx, 'product']
            
df_families.fillna(0,inplace=True)
df_families.iloc[-1,0] = df_families.iloc[-1, 2:-2].sum(axis=0)

In [ ]:
idx = pd.IndexSlice
prod_columns = [item for item in abundant_products]
prod_columns.append('Other')
for col in prod_columns:
    df_genome_data.loc[:,col] = 0
for pat_id in df_genome_data.index:
    genus_name = pat_genus_dict[pat_id]
    if genus_name in abundant_genera:
        df_genome_data.loc[pat_id, 'genus_cat'] = genus_name
    else:
        df_genome_data.loc[pat_id, 'genus_cat'] = 'Other'
        
    if pat_id in df_clusters.index.get_level_values(3).tolist():
        no_of_clusters = df_clusters.loc[idx[:,:,:,pat_id,:,:,:],:].index.get_level_values(5).tolist()[0]
        df_genome_data.loc[pat_id, 'no_of_clusters'] = no_of_clusters
        product_cat_list = df_clusters.loc[idx[:,:,:,pat_id,:,:,:],'product_cat'].tolist()
        for product_cat in product_cat_list:
            df_genome_data.loc[pat_id, product_cat] = df_genome_data.loc[pat_id, product_cat] + 1
    else:
        df_genome_data.loc[pat_id, 'no_of_clusters'] = 0
        
# Reindexing
df_genome_data = df_genome_data.reindex(columns= extracted_columns + ['genus_cat', 'no_of_clusters'] + prod_columns)
df_genome_data.sort_values('genus_cat', inplace=True)

genome_product_map = df_genome_data.iloc[:,6:]
# Colors according to genera
# Prepare a vector of color mapped to the 'cyl' column
my_palette = dict(zip(df_genome_data.genus_cat.unique(), sns.color_palette("tab20", len(df_genome_data.genus_cat.unique()))))
row_colors = df_genome_data.genus_cat.map(my_palette)

reorder_index = []
genus_list = list(abundant_genera)
genus_list.append('Other')

for genus in genus_list:
    genus_block_ids = df_genome_data[df_genome_data.genus_cat == genus].index
    df_genus_block = genome_product_map.loc[genus_block_ids,:]
    r_block = sns.clustermap(df_genus_block, cmap=sns.color_palette("CMRmap_r",10), col_cluster=False, row_cluster=True, row_colors=row_colors, 
              linewidths=0, yticklabels=False)
    reorder_index = reorder_index + r_block.data2d.index.tolist()

In [ ]:
# All clusters vs genus
df_clusters_tmp = df_clusters
cluster_class_count = df_clusters_tmp.groupby('bigscape_class').count()
cluster_class_count = cluster_class_count.drop(['notes','product'], axis=1)
cluster_class_count = cluster_class_count.rename(index=str, columns={"contig_edge": "No_clusters"})

cluster_prod_count = df_clusters_tmp.groupby(['bigscape_class', 'product']).count()
cluster_prod_count = cluster_prod_count.drop(['notes'], axis=1)
cluster_prod_count = cluster_prod_count.rename(index=str, columns={"contig_edge": "No_clusters"})
group_names = cluster_class_count.index.tolist()
group_size = cluster_class_count.No_clusters.tolist()
subgroup_names= cluster_prod_count.index.get_level_values('product').tolist()
subgroup_size = cluster_prod_count.No_clusters.tolist()

# # Create colors
color_out =[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens, plt.cm.Dark2, plt.cm.Accent, plt.cm.Greys, plt.cm.CMRmap, plt.cm.viridis]

colors_group = []
colors_subgroup = []
count_out = 0

for big_class in cluster_prod_count.index.levels[0]:
    colors_group.append(color_out[count_out](0.8))
    
    sub_df = cluster_prod_count.loc[big_class]
    count_in = 0
    prod_len = sub_df.shape[0]
    colorid_array = np.linspace(0.8, 0.5, num=prod_len)

    for prod in sub_df.index:
        colors_subgroup.append(color_out[count_out](colorid_array[count_in]))
        count_in = count_in + 1
    count_out = count_out + 1 
 
# First Ring (outside)
fig, ax = plt.subplots()
ax.axis('equal')

# colors_group = sns.color_palette("hls", cluster_count.shape[0])
# colors_subgroup = sns.color_palette("hls", cluster_count_genus.shape[0])

mypie, _ = ax.pie(group_size, radius=4, labels=group_size, labeldistance=1.05, rotatelabels=True, colors=colors_group)
plt.setp( mypie, width=1, edgecolor='white')
plt.rcParams.update({'font.size': 20})
# # Second Ring (Inside)
mypie2, _ = ax.pie(subgroup_size, radius=2.95, colors = colors_subgroup)
plt.setp( mypie2, width=2.9, edgecolor='white')
# plt.tight_layout()
plt.margins(0,0)
ax.add_artist(plt.legend(group_names, bbox_to_anchor=(-0.7, 1.),
           fontsize=14))
# ax.add_artist(plt.legend(subgroup_names, bbox_to_anchor=(-1.5, 1.),
#            fontsize=14))

# # # show it
plt.show()

In [ ]:
# TMP 
df_clusters_tmp = df_clusters
cluster_class_count = df_clusters_tmp.groupby('bigscape_class').count()
cluster_class_count = cluster_class_count.drop(['notes','product'], axis=1)
cluster_class_count = cluster_class_count.rename(index=str, columns={"contig_edge": "No_clusters"})

cluster_prod_count = df_clusters_tmp.groupby(['bigscape_class', 'product_cat']).count()
cluster_prod_count = cluster_prod_count.drop(['notes'], axis=1)
cluster_prod_count = cluster_prod_count.rename(index=str, columns={"contig_edge": "No_clusters"})

cluster_fam_count = df_clusters_tmp.groupby(['bigscape_class', 'product_cat', 'family_id']).count()
cluster_fam_count = cluster_fam_count.drop(['notes'], axis=1)
cluster_fam_count = cluster_fam_count.rename(index=str, columns={"contig_edge": "No_clusters"})

group_names = cluster_class_count.index.tolist()
group_size = cluster_class_count.No_clusters.tolist()
subgroup_names= cluster_prod_count.index.get_level_values('product_cat').tolist()
subgroup_size = cluster_prod_count.No_clusters.tolist()
subsubgroup_names= cluster_fam_count.index.get_level_values('product_cat').tolist()
subsubgroup_size = cluster_fam_count.No_clusters.tolist()
 
# # Create colors
# a, b, c=[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens]
 
# # First Ring (outside)
fig, ax = plt.subplots()
ax.axis('equal')

color_out =[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens, plt.cm.Dark2, plt.cm.Accent, plt.cm.Greys, plt.cm.CMRmap, plt.cm.viridis]

colors_group = []
colors_subgroup = []
colors_subsubgroup = []
count_out = 0

for big_class in cluster_prod_count.index.levels[0]:
    colors_group.append(color_out[count_out](0.8))
    
    sub_df = cluster_prod_count.loc[big_class]
    count_in = 0
    prod_len = sub_df.shape[0]
    colorid_array = np.linspace(0.8, 0.5, num=prod_len)

    for prod in sub_df.index:
        colors_subgroup.append(color_out[count_out](colorid_array[count_in]))
        
        
        subsub_df = cluster_fam_count.loc[big_class, prod]
        count_in_in = 0
        fam_len = subsub_df.shape[0]
        colorid_array_fam = np.linspace(0.8, 0.5, num=fam_len)

        for fam in subsub_df.index:
            colors_subsubgroup.append(color_out[count_out](colorid_array_fam[count_in_in]))
            count_in_in = count_in_in + 1
        count_in = count_in + 1
    
    count_out = count_out + 1 
    
                
# colors_subgroup = sns.color_palette("hls", cluster_prod_count.shape[0])

plt.rcParams.update({'font.size': 25})

# First Ring (Outside)
mypie, _ = ax.pie(group_size, radius=4 , colors=colors_group)
plt.setp( mypie, width=1, edgecolor='white')
ax.add_artist(plt.legend(list(zip(group_names,group_size)), bbox_to_anchor=(-1., 1.2),
           fontsize=18, title="Cluster class"))


# Second Ring (Inside)
mypie2, _ = ax.pie(subgroup_size, radius=2.8, colors=colors_subgroup)
plt.setp( mypie2, width=0.8, edgecolor='white')
# plt.tight_layout()
# ax.add_artist(plt.legend(list(zip(subgroup_names,subgroup_size)), bbox_to_anchor=(2, 1),
#            fontsize=18, title="Cluster product"))


# Third Ring (Inside)
mypie3, _ =  ax.pie(subsubgroup_size, radius=1.8, colors=colors_subsubgroup)
plt.setp( mypie3, width=1.8, edgecolor='white')
# plt.tight_layout()
plt.margins(0,0)


# show it
plt.show()

In [ ]:
# Analyse BiGSCAPE output
cut_offs = ['0.10', '0.20', '0.30', '0.50', '0.70', '1.00']
df_bigscape_gcfs = pd.DataFrame(columns= cut_offs)
net_path = '/home/omkar/Projects/panGenome/data/entero_project/bigscape_as4/network_files/2019-02-27_11-01-30_hybrids_glocal/'
# clust_types = [f for f in os.listdir(net_path) if '.tsv' not in f]
clust_types = ['Others', 'RiPPs']


for f in clust_types:
    for cut_off_id in cut_offs:
        csv_path = os.path.join(net_path, f, f + '_clustering_c' + cut_off_id + '.tsv')
        df_cutoff_gcf = pd.read_table(csv_path)
        for df_idx in df_cutoff_gcf.index:
            bgc_id = df_cutoff_gcf.loc[df_idx, '#BGC Name']
            fam_id = df_cutoff_gcf.loc[df_idx, 'Family Number']
            df_bigscape_gcfs.loc[bgc_id, cut_off_id] = fam_id
            df_bigscape_gcfs.loc[bgc_id, 'Cluster Type'] = f
            
        

In [ ]:
df_counts_gcf = df_bigscape_gcfs_type.groupby(['1.00', '0.70', '0.50', '0.30', '0.20','0.10']).count()
df_counts_gcf_rm_single = df_counts_gcf[df_counts_gcf['Cluster Type'] > 1]
# df_counts_gcf_rm_single.sort_values(by= 'Cluster Type', ascending= False)

In [ ]:
# Figure 2

In [ ]:
cluster_genus_dict = dict(zip(df_clusters.index.get_level_values(11) , df_clusters.index.get_level_values(1)))
cluster_product_dict = dict(zip(df_clusters.index.get_level_values(11) , df_clusters.bigscape_class))
cluster_type_dict = dict(zip(df_clusters.index.get_level_values(11) , df_clusters['product']))
cluster_mibig_dict = dict(zip(df_bigscape_nodes.index, df_bigscape_nodes.Organism))

genus_subgraph_dict = dict()
product_subgraph_dict = dict()
mibig_subgraph_dict = dict()

for subgraph_idx in range(len(family_graphs)):
    subgraph = family_graphs[subgraph_idx]
    
    if len(subgraph.node) > 100:
        genus_subgraph = []
        product_subgraph = []
        type_subgraph = []
        mibig_names = []
        for node in subgraph.node:
            nodes_large_subgraphs.append(node)
            if 'BGC' not in node:
                genus_subgraph.append(cluster_genus_dict[node])
                product_subgraph.append(cluster_product_dict[node])
                type_subgraph.append(cluster_type_dict[node])
            else:
                genus_subgraph.append('MIBIG')
                mibig_names.append(cluster_mibig_dict[node])
                
        genus_subgraph_dict[subgraph_idx] = genus_subgraph
        product_subgraph_dict[subgraph_idx] = product_subgraph
        mibig_subgraph_dict[subgraph_idx] = mibig_names
        cluster_type_dict[subgraph_idx] = type_subgraph


In [ ]:
for subgraph_idx in range(len(genus_subgraph_dict)):
    subgraph = family_graphs[subgraph_idx]
    
    genus_subgraph = genus_subgraph_dict[subgraph_idx]
    product_subgraph = product_subgraph_dict[subgraph_idx]
    type_subgraph = cluster_type_dict[subgraph_idx]
    mibig_names = mibig_subgraph_dict[subgraph_idx]
    
    no_nodes = len(subgraph.nodes)
    no_edges = len(subgraph.edges)
    cluster_product = Counter(product_subgraph).most_common(1)[0][0]
    cluster_type = Counter(type_subgraph).most_common(1)[0][0]
    
    sns.set(context='poster', style='white', palette='deep', font='sans-serif', font_scale=1.4)
    # # First Ring (outside)
    fig, ax = plt.subplots()
    ax.axis('equal')

    my_palette['MIBIG'] = 'black'

    group_names = list(Counter(genus_subgraph).keys())
    group_size = list(Counter(genus_subgraph).values())

    group_colors = []
    for genus_id in group_names:
        group_colors.append(my_palette[genus_id])

    # label_dist_2 = (1.11, 1.11,1.11,1.11,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05,1.05)
#     labels = [group_names[cnt] + ', ' + str(group_size[cnt]) for cnt in range(len(group_names))]
    # explode = (0,0,0,0.4,0.4,0,0,0,0.4,0.4,1.2,0.4,1.2,0,0,0)
    mypie1, autotexts1 = ax.pie(group_size, radius=2,labeldistance=1, rotatelabels=True, colors=group_colors, textprops={'fontsize': 15, 'fontweight':'bold'})
    # mypie2, autotexts2 = ax.pie(group_size, radius=4, labels=cluster_count_avg_annot, labeldistance=1.1, explode=explode,rotatelabels=True, colors=group_colors, textprops={'fontsize': 15, 'fontweight':'bold'})
    
    
#     if not mibig_names:
#         text_label = 'Family-' + str(subgraph_idx + 1) + '\nCluster-' + cluster_type + '\nNo MIBIG clusters'+ '\n#Nodes- ' + str(no_nodes) + '\n#Edges- ' + str(no_edges)
#     else:
#         text_label = 'Family-' + str(subgraph_idx + 1) + '\nCluster-' + cluster_type + '\n' + ','.join(mibig_names)+ '\n#Nodes- ' + str(no_nodes) + '\n#Edges- ' + str(no_edges)
    
    if not mibig_names:
        text_label ='No MIBIG clusters'+ '\n#Nodes- ' + str(no_nodes) + '\n#Edges- ' + str(no_edges)
    else:
        text_label = ','.join(mibig_names)+ '\n#Nodes- ' + str(no_nodes) + '\n#Edges- ' + str(no_edges)
    
    plt.text(-1.8,-0.5, text_label, bbox=dict(facecolor='white', alpha=0.5), fontsize=28, fontweight='bold')

    for autotext in autotexts1:
        autotext.set_color('blue')

    # for autotext in autotexts2:
    #     autotext.set_color('red')

    plt.setp( mypie1, width=2, edgecolor='white')
    fig_path = os.path.join(data_dir, 'cytoscape', 'Output', str(subgraph_idx+1) + '_family_dist.png')
    plt.savefig(fig_path,pad_inches = 0.1, bbox_inches = 'tight')

In [ ]:
# scat_data = df_singleton[['Genome Length', 'no_of_clusters', 'GC Content']].astype(float)
# scat_data['Genome Length'] = round(scat_data['Genome Length']/1000000,1)
# scat_data['GC Content'] = round(scat_data['GC Content'],1)
# scat_data['genus_cat'] = df_genome_data.genus_cat
# df = scat_data

# df_counts = df.groupby(['GC Content', 'no_of_clusters','genus_cat']).size().reset_index(name='counts')
# df_counts = df.groupby(['Genome Length', 'no_of_clusters','genus_cat']).size().reset_index(name='counts')

df_counts = scat_data
cnt_colors = df_counts.genus_cat.map(my_palette)

# Create Fig and gridspec
fig = plt.figure(figsize=(16, 10), dpi= 80)
grid = plt.GridSpec(4, 4, hspace=0.6, wspace=0.2)

# Define the axes
ax_main = fig.add_subplot(grid[:-1, :-1])
# ax_right = fig.add_subplot(grid[:-1, -1], yticklabels=[])
# ax_bottom = fig.add_subplot(grid[-1, 0:-1], xticklabels=[])

# Scatterplot on main ax
# ax_main.scatter('Genome Length', 'no_of_clusters', data=df_counts, s=df_counts.counts*40, c=cnt_colors, alpha = 0.55, 
#                 edgecolors='gray', linewidths=.5)
ax_main.scatter('genus_cat', 'product_cat', data=df_counts, s=df_counts.counts*33, label=df_counts.counts.tolist(), c=cnt_colors, alpha = 0.55,
                edgecolors='gray', linewidths=.5)
# sns.stripplot('Genome Length', 'no_of_clusters', data=df, c=row_colors, jitter=0.25, ax=ax_main)

# histogram on the right
# ax_bottom.hist(df['GC Content'], 40, histtype='stepfilled', orientation='vertical', color='deeppink')
# ax_bottom.hist(df['Genome Length'], 40, histtype='stepfilled', orientation='vertical', color='deeppink')

# ax_bottom.invert_yaxis()

# histogram in the bottom
# ax_right.hist(df['no_of_clusters'], int(max(df.no_of_clusters) + 1), histtype='stepfilled', orientation='horizontal', color='deeppink')

# Decorations
ax_main.set(title='Singleton clusters vs genera', 
            xlabel='Genera', ylabel='Cluster class')
# ax_main.set(title='No. of clusters vs Genome Length', 
#             xlabel='Genome Length (Mb)', ylabel='No of clusters per genome')
ax_main.title.set_fontsize(24)
plt.xticks(rotation='vertical')
plt.legend()
# ax_bottom.set(ylabel='No of genomes')
# ax_right.set(xlabel='No of genomes')

# Annotate 
# from matplotlib.patches import Ellipse
# el = Ellipse((5.4, 21), 1.2, 6, angle=0,facecolor='g',alpha=0.2)

# ax_main.add_artist(el)
# el.set_clip_box(ax_main.bbox)
# ax_main.annotate('Photorhabdus()',
#             xy=(5.8, 22),      # theta, radius
#             xytext=(6.5, 22),   # theta, radius
#             arrowprops=dict(facecolor='black', shrink=0.04),
#             horizontalalignment='left',
#             verticalalignment='bottom', fontsize=16,
#             clip_on=True)  # clip to the axes bounding box


for item in ([ax_main.xaxis.label, ax_main.yaxis.label] + ax_main.get_xticklabels() + ax_main.get_yticklabels()):
    item.set_fontsize(18)

# for item in ([ax_right.xaxis.label, ax_right.yaxis.label] + ax_right.get_xticklabels() + ax_right.get_yticklabels()):
#     item.set_fontsize(18)
    
# for item in ([ax_bottom.xaxis.label, ax_bottom.yaxis.label] + ax_bottom.get_xticklabels() + ax_bottom.get_yticklabels()):
#     item.set_fontsize(18)
plt.show()

In [ ]:
df_clusters[df_clusters.family_id == 'singleton'].groupby(['genus_cat','product_cat']).count().contig_edge
df_singleton = df_clusters[df_clusters.family_id == 'singleton']
df_singleton['genus_cat'] = df_singleton.index.get_level_values(1)
df_singleton = df_singleton[['genus_cat','product_cat','contig_edge']]
df_singleton.index = df_singleton.index.get_level_values(11)
scat_data = df_singleton.groupby(['genus_cat','product_cat']).size().reset_index(name='counts')

In [ ]:
len(dict(zip(df_clusters.mibig_ids , df_clusters.family_id)))
# df_ecoli = df_clusters[df_clusters.index.get_level_values(0) == 'Serratia']
df_ecoli = df_clusters
df_ecoli = df_ecoli[df_ecoli.product_cat == 'thiopeptide']

# df_ecoli = df_ecoli[df_ecoli.index.get_level_values(5) == 8]

# df_ecoli.index.get_level_values(5)
df_cnt = df_ecoli.groupby(['family_id','genus_cat']).count().sort_values('product', ascending=False)
df_cnt = df_ecoli.groupby('family_id').count().sort_values('product', ascending=False)

# df_cnt.product_cat.hist(bins = 6)
# df_cnt[df_cnt.index.get_level_values(1)=='Serratia']
df_cnt

In [ ]:
# Nodes in subnetworks with at least 10 clusters
nodes_large_subgraphs = []
nodes_small_subgraphs = []
for subgraph in family_graphs:
    if len(subgraph.node) > 100:
        for node in subgraph.node:
            nodes_large_subgraphs.append(node)
    else:
        for node in subgraph.node:
            nodes_small_subgraphs.append(node)

# Update df_union network accordingly
df_network_union_large = df_network_union_filter_singles
df_network_union_small = df_network_union_filter_singles
idx_large = []
idx_small = []
for df_idx in df_network_union_filter_singles.index:
    if df_network_union_filter_singles.loc[df_idx, 'Clustername 1'] in nodes_large_subgraphs or df_network_union_filter_singles.loc[df_idx, 'Clustername 2'] in nodes_large_subgraphs:
        idx_large.append(df_idx)
#         print(df_idx)
    else:
        idx_small.append(df_idx)
    
df_network_union_large = df_network_union_filter_singles.loc[idx_large,:]
df_network_union_small = df_network_union_filter_singles.loc[idx_small,:]
df_network_union_large.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_large.csv'))
df_network_union_small.to_csv(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_small.csv'))

df_network_union_large.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_large.p'))
df_network_union_small.to_pickle(os.path.join(data_dir, 'cytoscape','Input', 'df_network_union_small.p'))